In [11]:
using JuMP, MosekTools
using Plots, Combinatorics, LinearAlgebra
using DelimitedFiles
using PoissonRandom
using NPZ
using Ipopt
include("QuantumNPA/QuantumNPA.jl");
using .QuantumNPA
include("dist_builder.jl");

In [2]:
using Base.Threads
nthreads()

1

# Common definitions

In [3]:

function get_proj(obs)
    vals, vecs = eigen(obs)
    if vals[1] ≈ -1 && vals[2] ≈ 1
        return [vecs[:,2]*vecs[:,2]', vecs[:,1]*vecs[:,1]']
    elseif vals[1] ≈ 1 && vals[1] ≈ -1
        return [vecs[:,1]*vecs[:,1]', vecs[:,2]*vecs[:,2]']
    else
        throw("Non dichotomic observable: Eigenvalues: $vals.")
    end
end

function _cyclic_mon(P, n, x1, x2)
    m = Id
    for (i, x) in enumerate(Iterators.cycle([x1, x2]))
        m *= P[1, x]
        i ≥ n && break
    end
    return m
end

function generate_projective_scvar(P, N, x1, x2; tail=Id, swapped=true, var_symbols=nothing)
    vars = []
    for n=2:N
        Pn12 = _cyclic_mon(P, n, x1, x2)*tail
        if isnothing(var_symbols)
            αn12 = scalarfactor(Pn12)
        else
            αn12 = scalarfactor("$(var_symbols[1])_$(n)", Pn12)
        end
        push!(vars, αn12)
        if swapped 
            Pn21 = _cyclic_mon(P, n, x2, x1)*tail
            if isnothing(var_symbols)
                αn21 = scalarfactor(Pn21)
            else
                αn21 = scalarfactor("$(var_symbols[2])_$(n)", Pn21)
            end
            push!(vars, αn21)
        end
    end
    return vars
end

function generate_projective_scvar(P, N; tail=Id, swapped=true, var_symbols=nothing)
    num_settings = size(PA)[2]
    vars = []
    for (x1, x2) in combinations(1:num_settings, 2)
        if !isnothing(var_symbols)
            var_symbols = map(s->"$s$x1$x2", var_symbols)
        end
        push!(vars, generate_projective_scvar(P, N, x1, x2, tail=tail, swapped=swapped, var_symbols=var_symbols)...)
    end
    return vars
end

"""Search for the minimum value inside `interval` for which `f(x)` is true.
Returns an interval such that `x_min ∈ [x1, x2]`.""" 
function min_binary_search(f::Function, interval::Tuple{Float64, Float64}, N::Int; verbose=false)
    min_int = [interval...]
    for i=1:N
        x = sum(min_int)/2
        verbose && print("\rChecking interval $min_int")
        if f(x)
            min_int[2] = x
        else
            min_int[1] = x
        end
    end
    verbose && println()
    return min_int
end




min_binary_search

## Definitions

### Strategy definitions

In [1]:
ide = [1 0; 0 1]
II = kron(ide, ide)

# State cos(θ)|00> + sin(θ)|11>
ψ = θ -> [cos(θ); 0; 0; sin(θ)]
ρwer = (θ,v) -> v*(ψ(θ) * ψ(θ)') + (1-v)*II/4

function gen_abxy_bell(v, θ; 
                 ρ=ρwer, 
                 obsA=[get_proj(σ[3]), get_proj(σ[1])], 
                 obsB=[get_proj((σ[1]+σ[3])/√2), get_proj((-σ[1]+σ[3])/√2)])
    p = zeros(2,2,2,2)
    state = ρ(θ,v)
    for (a,b,x,y) in Iterators.product(1:2, 1:2, 1:2, 1:2)
        p[a,b,x,y] = real(tr(state * kron(obsB[y][b], obsA[x][a])))
    end
    return p
end

# Tilted Tony
μ(α, θ) = atan(sin(2θ/α))
PA_atilted = [get_proj(σ[3]),
              get_proj(σ[1])]
PB_atilted(α, θ) = [get_proj(cos(μ(α,θ))*σ[3] + sin(μ(α,θ))*σ[1]),
                    get_proj(cos(μ(α,θ))*σ[3] - sin(μ(α,θ))*σ[1])]

function gen_abxy_atilted(v, θ, α)
    return gen_abxy_bell(v, θ; 
                 ρ=ρwer, 
                 obsA=PA_atilted, 
                 obsB=PB_atilted(α, θ))
end

function bell_atilted_from_dist(p, α, β)
    ab(x,y) = sum(p[a,b,x,y] * (-1)^(a+b) for a=1:2, b=1:2)
    a(x) = sum(p[a,b,x,1] * (-1)^(a) for a=1:2, b=1:2)
    return β*a(1) + α*ab(1,1) + ab(2,1) + α*ab(1,2) - ab(2,2)
end

# Tilted Wooltorton 1
PA_wtilted1(δ) = [get_proj(σ[3]),
                  get_proj(-sin(δ)*σ[3] + cos(δ)*σ[1])]
PB_wtilted1(δ) = [get_proj(σ[1]),
                  get_proj(cos(δ)*σ[3] - sin(δ)*σ[1])]

function gen_abxy_wtilted1(v, δ; θ=π/4)
    return gen_abxy_bell(v, θ;
                 ρ=ρwer, 
                 obsA=PA_wtilted1(δ), 
                 obsB=PB_wtilted1(δ))
end

function p_bilocality_22_wtilted(δ; θ=π/4, v=1, id=1)
    A = ΠA_wtilted1(δ)
    C = ΠC_wtilted1(δ)
    B = [[kron(B1[1], B2[1]) + kron(B1[2], B2[2]),
          kron(B1[1], B2[2]) + kron(B1[2], B2[1])]
        for (B1, B2) in zip(C, A)]
    ρ = kron(ρ_bell(v, θ=θ, id=id),
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end

function bell_wtilted1_from_dist(p, δ)
    ab(x,y) = sum(p[a,b,x,y] * (-1)^(a+b) for a=1:2, b=1:2)
    return ab(1,1) + (ab(1,2) + ab(2,1))/sin(δ) - ab(2,2)/cos(2δ)
end

# Tilted Wooltorton 2
PA_wtilted2(γ) = [get_proj(σ[3]),
                  get_proj(cos(2π/3 - 2γ)*σ[3] + sin(2π/3 - 2γ)*σ[1])]
PB_wtilted2(γ) = [get_proj(sin(3γ)*σ[3] + cos(3γ)*σ[1]),
                  get_proj(cos(π/6 + γ)*σ[3] - sin(π/6 + γ)*σ[1])]

function gen_abxy_wtilted2(v, γ; θ=π/4)
    return gen_abxy_bell(v, θ;
                 ρ=ρwer, 
                 obsA=PA_wtilted2(γ), 
                 obsB=PB_wtilted2(γ))
end

function bell_wtilted2_from_dist(p, γ)
    ab(x,y) = sum(p[a,b,x,y] * (-1)^(a+b) for a=1:2, b=1:2)
    c = 4*cos(γ+π/6)^2-1 
    return ab(1,1) + c*(ab(1,2) + ab(2,1) - ab(2,2))
end

# Behavior constraints

function get_bconstraints(p)
    return [PA[1,1] - Id*sum(p[1,:,1,1]),
            PA[1,2] - Id*sum(p[1,:,2,1]),
            PB[1,1] - Id*sum(p[:,1,1,1]),
            PB[1,2] - Id*sum(p[:,1,1,2]),
            PA[1,1]*PB[1,1] - Id*p[1,1,1,1],
            PA[1,1]*PB[1,2] - Id*p[1,1,1,2],
            PA[1,2]*PB[1,1] - Id*p[1,1,2,1],
            PA[1,2]*PB[1,2] - Id*p[1,1,2,2]]
end

LoadError: UndefVarError: `σ` not defined

# Quantum Bilocality

In [ ]:
function generate_bilocal_eq_constraints(p, PA::Matrix{Any}, PB::Matrix{Any}, PC::Matrix{Any})
    nA, nX = size(PA)
    nC, nZ = size(PC)
    nB, nY = size(PB)
    eqs = [PA[a,x]*PB[b,y]*PC[c,z] - Id*p[a,b,c,x,y,z] 
        for (a,b,c,x,y,z) in Iterators.product(1:nA, 1:nB, 1:nC, 1:nX, 1:nY, 1:nZ)]
    eqs = [eqs...];
    
    return eqs
end

function generate_bilocal_eq_constraints(p, PA::Matrix{Any}, PB::Vector{Any}, PC::Matrix{Any})
    nA, nX = size(PA)
    nC, nZ = size(PC)
    nB = size(PB)[1]
    eqs = [PA[a,x]*PB[b]*PC[c,z] - Id*p[a,b,c,x,z]
        for (a,b,c,x,z) in Iterators.product(1:nA, 1:nB, 1:nC, 1:nX, 1:nZ)]
    eqs = [eqs...];
    
    return eqs
end

function bilo_is_feasible(p, PA, PB, PC; commutative=false, level=2)
    eqs = generate_bilocal_eq_constraints(p, PA, PB, PC)
    model = npa2jump(Id, level, se, eq=eqs, solver=Mosek.Optimizer, verbose=false)
    optimize!(model)
    return !is_solved_and_feasible(model)
end

function bilo_pguess(p, PA, PB, PC, PE, se; level=2, x=1, y=1, z=1, verbose=false)
    nA, nX = size(PA)
    nC, nZ = size(PC)
    nB = size(PB)[1]
    nE = size(PE)[1]
    
    eqs = generate_bilocal_eq_constraints(p, PA, PB, PC)
    
    # Select the correct pguess (guessing B or not) depending on the PE dimension
    if nE > nA*nC
        if length(size(PB)) > 1
            pguess = sum(PA[a,x]*PB[b,y]*PC[c,z]*PE[(a-1) + (b-1)*nA + (c-1)*nA*nB + 1] for (a,b,c) in Iterators.product(1:nA, 1:nB, 1:nC))
        else
            pguess = sum(PA[a,x]*PB[b]*PC[c,z]*PE[(a-1) + (b-1)*nA + (c-1)*nA*nB + 1] for (a,b,c) in Iterators.product(1:nA, 1:nB, 1:nC))
        end
    else
        pguess = sum(PA[a,x]*PC[c,z]*PE[(a-1) + (c-1)*nA + 1] for (a,c) in Iterators.product(1:nA, 1:nC))
    end
    
    model = npa2jump(pguess, level, se, eq=eqs, solver=Mosek.Optimizer, verbose=verbose)
    optimize!(model)
    return objective_value(model)
end

function bilo_pguess(p, PA, PB, PC, PE, PF, se; 
        level=2, x=1, y=1, z=1, 
        fb0 = b -> b%2,
        fb1 = b -> b÷2,
        verbose=false)
    nA, nX = size(PA)
    nC, nZ = size(PC)
    nB = size(PB)[1]
    nE = size(PE)[1]
    nF = size(PF)[1]
    
    eqs = generate_bilocal_eq_constraints(p, PA, PB, PC)
    
    # Select the correct pguess (guessing B or not) depending on the PE dimension
    if nE*nF > nA*nC
        if nB == 4
            if length(size(PB)) > 1
                pguess = sum(PA[a,x] * PB[b,y] * PC[c,z] * PE[(a-1)+fb0(b-1)*nA+1] * PF[(c-1)+fb1(b-1)*nC+1] 
                    for (a,b,c) in Iterators.product(1:nA, 1:nB, 1:nC))
            else
                pguess = sum(PA[a,x] * PB[b] * PC[c,z] * PE[(a-1)+fb0(b-1)*nA+1] * PF[(c-1)+fb1(b-1)*nC+1] 
                    for (a,b,c) in Iterators.product(1:nA, 1:nB, 1:nC))
            end
        elseif nB == 2
            if nE == 2 && nF == 4
                pguess = sum(PA[a,x] * PB[b] * PC[c,z] * PE[a] * PF[(c-1)+(b-1)*nC+1] 
                    for (a,b,c) in Iterators.product(1:nA, 1:nB, 1:nC))
            elseif nE == 4 && nF == 2
                pguess = sum(PA[a,x] * PB[b] * PC[c,z] * PE[(a-1)+(b-1)*nA+1] * PF[c]
                    for (a,b,c) in Iterators.product(1:nA, 1:nB, 1:nC))
            else
                throw("Invalid Eavesdropper dimensions")
            end 
        else
            throw("Only 1 or 2 bits central node is supported")
        end
    else
        pguess = sum(PA[a,x] * PC[c,z] * PE[a] * PF[c] 
            for (a,c) in Iterators.product(1:nA, 1:nC))
    end
    
    model = npa2jump(pguess, level, se, eq=eqs, solver=Mosek.Optimizer, verbose=verbose)
    optimize!(model)
    return objective_value(model)
end

function _ef_fun_generator()
    function _get_πfuncs(π)
        return b -> π[b+1] % 2, b -> π[b+1] ÷ 2
    end
    return map(_get_πfuncs, permutations([0,1,2,3]))
end

# No settings on B
function p_bilocality(A::Vector{Vector{Matrix{T}}}, 
                      B::Vector{Matrix{T}}, 
                      C::Vector{Vector{Matrix{T}}}, 
                      ρ::Matrix{T}) where T <: Complex
    num_outs = (length(A[1]), length(B), length(C[1]))
    num_sets = (length(A), length(C))
    p = zeros(num_outs..., num_sets...)
    for s in Iterators.product((1:n for n in num_sets)...)
        for o in Iterators.product((1:n for n in num_outs)...)
            ABC = kron(A[s[1]][o[1]], B[o[2]], C[s[2]][o[3]])
            p[o..., s...] = real(tr(ABC*ρ))
        end
    end
    return p
end

# B with settings
function p_bilocality(A::Vector{Vector{Matrix{T}}}, 
                      B::Vector{Vector{Matrix{T}}}, 
                      C::Vector{Vector{Matrix{T}}}, 
                      ρ::Matrix{T}) where T <: Complex
    num_outs = (length(A[1]), length(B[1]), length(C[1]))
    num_sets = (length(A), length(B), length(C))
    p = zeros(num_outs..., num_sets...)
    for s in Iterators.product((1:n for n in num_sets)...)
        for o in Iterators.product((1:n for n in num_outs)...)
            ABC = kron(A[s[1]][o[1]], B[s[2]][o[2]], C[s[3]][o[3]])
            p[o..., s...] = real(tr(ABC*ρ))
        end
    end
    return p
end

ρ2_bell(v1, v2; id=4) = kron(ρ_bell(v1, id=id), ρ_bell(v2, id=id))

σt(α1,α2) = [get_proj(sin(α1)*σ[3] + cos(α1)*σ[1]),
             get_proj(sin(α2)*σ[3] + cos(α2)*σ[1])]

ρ_bell(v; θ=π/4, id=4) = v*bell_state(id, θ=θ) + (1-v)*eye(4)/4

μ(α, θ) = atan(sin(2θ/α))
Π_atilted(α, θ) = [get_proj(cos(μ(α,θ))*σ[3] + sin(μ(α,θ))*σ[1]),
                  get_proj(cos(μ(α,θ))*σ[3] - sin(μ(α,θ))*σ[1])]

function p_bilocality_abell(;θ=π/4, v=1, α=1, id=4)
    
    B = [bell_state(k) for k=1:4]
    A = σt(0, π/2)
    C = Π_atilted(α, θ)
    ρ = kron(ρ_bell(v, θ=θ, id=id), 
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end

ΠA_wtilted1(δ) = [get_proj(σ[3]),
                  get_proj(-sin(δ)*σ[3] + cos(δ)*σ[1])]
ΠC_wtilted1(δ) = [get_proj(σ[1]),
                  get_proj(cos(δ)*σ[3] - sin(δ)*σ[1])]

function p_bilocality_wbell1(δ; θ=π/4, v=1, id=4)
    B = [bell_state(k) for k=1:4]
    A = ΠA_wtilted1(δ)
    C = ΠC_wtilted1(δ)
    ρ = kron(ρ_bell(v, θ=θ, id=id), 
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end

ΠA_wtilted2(γ) = [get_proj(σ[3]),
                  get_proj(cos(2π/3 - 2γ)*σ[3] + sin(2π/3 - 2γ)*σ[1])]
ΠC_wtilted2(γ) = [get_proj(sin(3γ)*σ[3] + cos(3γ)*σ[1]),
                  get_proj(cos(π/6 + γ)*σ[3] - sin(π/6 + γ)*σ[1])]

function p_bilocality_wbell2(γ; θ=π/4, v=1, id=1)
    B = [bell_state(k) for k=1:4]
    A = ΠA_wtilted2(γ)
    C = ΠC_wtilted2(γ)
    ρ = kron(ρ_bell(v, θ=θ, id=id), 
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end


rotated_BSM(θ) = [bell_state(i, θ=θ) for i=1:4]

function p_bilocality_abellBSM(φ; θ=π/4, v=1, α=1, id=4)
    B = rotated_BSM(φ)
    A = σt(0, π/2)
    C = Π_atilted(α, θ)
    ρ = kron(ρ_bell(v, θ=θ, id=id), 
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end

function p_bilocality_tiltedBSM1(δ, φ; θ=π/4, v=1, id=1)
    B = rotated_BSM(φ)
    A = ΠA_wtilted1(δ)
    C = ΠC_wtilted1(δ)
    ρ = kron(ρ_bell(v, θ=θ, id=id), 
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end

function p_bilocality_tiltedBSM2(γ, φ; θ=π/4, v=1, id=1)
    B = rotated_BSM(φ)
    A = ΠA_wtilted2(γ)
    C = ΠC_wtilted2(γ)
    ρ = kron(ρ_bell(v, θ=θ, id=id), 
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end

ΠA_chsh = [get_proj(σ[3]),
              get_proj(σ[1])]
ΠC_chsh = [get_proj((σ[3] + σ[1])/√2),
           get_proj((σ[3] - σ[1])/√2)]

function p_bilocality_24_chsh_rotBSM(φ; θ=π/4, v=1, α=1, id=1)
    B = [rotated_BSM(π/4), rotated_BSM(φ)]
    #A = ΠA_chsh
    A = Π_atilted(α, θ)
    C = Π_atilted(α, θ)
    ρ = kron(ρ_bell(v, θ=θ, id=id),
             ρ_bell(v, θ=θ, id=id))
    return p_bilocality(A, B, C, ρ)
end

function p_bilocality_24_wtiltedBSM(δ, θ; θsrc=π/4, v=1, id=1)
    B = [rotated_BSM(π/4), rotated_BSM(θ)]
    
    A = ΠA_wtilted1(δ)
    C = ΠC_wtilted1(δ)
    ρ = kron(ρ_bell(v, θ=θsrc, id=id),
             ρ_bell(v, θ=θsrc, id=id))
    return p_bilocality(A, B, C, ρ)
end;

# Standard strategies

We define "standard strategies" the measurement choices on the external nodes ($A$ and $C$) defined in Eq.(12) of the main text:
\begin{equation}
 A_{0,1} = C_{0,1} = \frac{\sigma_z + (-1)^{0,1} \sigma_x}{\sqrt{2}}
\end{equation}
We consider different possible eavesdropping strategies and measurements in the central node. We denote the possible choices of measurements in the central node with the number of settings and outcomes of the measurement:

- (1,4): Bell State Measurement (BSM)
- (2,2): $B_0 = \sigma_x \otimes \sigma_x$, $B_1 = \sigma_z \otimes \sigma_z$

Instead, the possible eavesdropping strategies are:
- Strong Eavesdropper (SE)
- Double Eavesdropper (DE)

(forse DAG corrispondenti)


## (1, 4) measurement strategy

### Definitions

#### A, B, C projectors

In [20]:
# Define projectors corresponding to the three parties measurements
PA = projector('A', 1:2, 1:2, full=true);
PB_14= projector('B', 1:4, 1, full=true);
PC = projector('C', 1:2, 1:2, full=true);

#### Strong-Eavesdropper projectors

In [21]:
# Define projectors corresponding to the eavesdropper measurements
PE2 = projector('D', 1:4, 1, full=true); # E guessing AC
PE4 = projector('D', 1:16, 1, full=true); # E guessing ABC


l = 5 # Order of the product of the scalar variables
con = separability_structure(["A", "B", "C", "D"], ("A","C")); # Separability constraints
vars = generate_projective_scvar(PA, l, swapped=true); # Scalar variables
se_14_strong = ScalarExtension(vars, con); # Scalar Extension

#### Double-Eavesdropper projectors

Guessing ABC

In [22]:
# Define projectors corresponding to the eavesdroppers measurements
PE2 = projector('D', 1:4, 1, full=true); # E guessing A,B1 (B1 = first bit of B's outcome)
PF2 = projector('E', 1:4, 1, full=true); # F guessing C,B2 (B2 = second bit of B's outcome)


con = separability_structure(["A", "B", "C", "D", "E"], ("A","C"), ("E", "D"), ("D", "C"), ("E", "A")); # Separability constraints

l=5 # Order of the product of the scalar variables

# Scalar variables
vars = [generate_projective_scvar(PA, l, swapped=true);
        generate_projective_scvar(PA, l, tail=PE2[1], swapped=true);
        generate_projective_scvar(PA, l, tail=PE2[2], swapped=true);
        generate_projective_scvar(PA, l, tail=PE2[3], swapped=true);
        [scalarfactor(PE2[i]) for i=1:3];
]
se_14_double_ABC = ScalarExtension(vars, con); # Scalar Extension

Guessing only A and C

In [23]:
PE1 = projector('D', 1:2, 1, full=true); # E guessing A
PF1 = projector('E', 1:2, 1, full=true); # F guessing C

con = separability_structure(["A", "B", "C", "D", "E"], ("A","C"), ("E", "D"), ("D", "C"), ("E", "A")); # Separability constraints

l=5 # Order of the product of the scalar variables

# Scalar variables
vars = [generate_projective_scvar(PA, l, swapped=true);
        generate_projective_scvar(PA, l, tail=PE1[1], swapped=true);
        [scalarfactor(PE1[1])];
]
se_14_double_AC = ScalarExtension(vars, con); # Scalar Extension

#### Measurements operators

In [24]:
B14 = [bell_state(k) for k=1:4];

A = σt(π/4, -π/4);
C = σt(π/4, -π/4);

### Strong Eavesdropper

#### Guessing A, B, C

In [13]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) # Visibilities of the state
pgs = ones(length(vs))


@threads for (i,v) ∈ collect(enumerate(vs))
    p = p_bilocality(A, B14, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_14, PC, PE4, se_14_strong, level=level, x=1, z=1) # Solution of the SDP problem

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.3751984846310349

#### Guessing A, C

In [17]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) # Visibilities of the state
pgs = ones(length(vs))


@threads for (i,v) ∈ collect(enumerate(v))
    p = p_bilocality(A, B14, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_14, PC, PE2, se_14_strong, level=level, x=1, z=1) # Solution of the SDP problem

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.3752320853300831

### Double-Eavesdropper

#### Guessing A, B, C

In [19]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) # Visibilities of the state
pgs = ones(length(vs))
@threads for (i,v) ∈ collect(enumerate(vs))
    p = p_bilocality(A, B14, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_14, PC, PE2, PF2, se_14_double_ABC, level=level, x=1, z=1) # Solution of the SDP problem

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.12507665621791897

#### Guessing A, C

In [20]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) 
pgs = ones(length(vs))
@threads for (i,v) ∈ collect(enumerate([1.0]))
    p = p_bilocality(A, B14, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_14, PC, PE1, PF1, se_14_double_AC, level=level, x=1, z=1)

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.3750652691063153

## (2, 2) measurement strategy

### Definitions

#### A, B, C projectors

In [22]:
# Define projectors corresponding to the three parties measurements
PA = projector('A', 1:2, 1:2, full=true);
PB_22 = projector('B', 1:2, 1:2, full=true);
PC = projector('C', 1:2, 1:2, full=true);

#### Strong-Eavesdropper projectors

In [23]:
# Define projectors corresponding to the eavesdropper measurements
PE2 = projector('D', 1:4, 1, full=true); # E guessing A, C
PE3 = projector('D', 1:8, 1, full=true); # E guessing A, B, C


l = 5 # Order of the product of the scalar variables
con = separability_structure(["A", "B", "C", "D"], ("A","C")); # Separability constraints
vars = generate_projective_scvar(PA, l, swapped=true); # Scalar variables 
se_22_strong = ScalarExtension(vars, con); # Scalar Extension 

#### Double-Eavesdropper projectors

Guessing ABC

In [24]:
# Define projectors corresponding to the eavesdroppers measurements
PE2 = projector('D', 1:4, 1, full=true) # E guessing A, B 
PF1 = projector('E', 1:2, 1, full=true) # F guessing C


con = separability_structure(["A", "B", "C", "D", "E"], ("A","C"), ("E", "D"), ("D", "C"), ("E", "A")); # Separability constraints

l=5 # Order of the product of the scalar variables

# Scalar variables
vars = [generate_projective_scvar(PA, l, swapped=true);
        generate_projective_scvar(PA, l, tail=PE2[1], swapped=true);
        generate_projective_scvar(PA, l, tail=PE2[2], swapped=true);
        generate_projective_scvar(PA, l, tail=PE2[3], swapped=true);
        [scalarfactor(PE2[i]) for i=1:3];
]
se_22_double_ABC = ScalarExtension(vars, con); # Scalar Extension

Guessing only A and C

In [25]:
PE1 = projector('E', 1:2, 1, full=true) # E guessing A
PF1 = projector('D', 1:2, 1, full=true) # F guessing C

con = separability_structure(["A", "B", "C", "D", "E"], ("A","C"), ("E", "D"), ("E", "C"), ("D", "A")); # Separability constraints

l=5 # Order of the product of the scalar variables

# Scalar variables
vars = [generate_projective_scvar(PA, l, swapped=true);
        generate_projective_scvar(PA, l, tail=PE1[1], swapped=true);
        [scalarfactor(PE1[1])];
]
se_22_double_AC = ScalarExtension(vars, con); # Scalar Extension

#### Measurements operators

Both the (2,2) measurement operators have two pairs of degenerate eigenvalues (and the corresponding eigenvectors)  $\lambda_{\mathrm{deg}} = \{-1,-1,1,1\}$ and $\Pi_{\mathrm{deg}} = \{ \Pi_{-1}^i, \Pi_{-1}^{ii}, \Pi_1^i, \Pi_1^{ii}  \}$. We are interested in the bitwise product of the separate meassurements outcomes, and the corresponding projectors consists in the ones that project onto the degenerate spaces, i.e $\Pi_{\mathrm{non-deg}} = \{ \Pi_{-1}^i + \Pi_{-1}^{ii}, \Pi_1^i + \Pi_1^{ii}  \}$. These are given by the function `get_proj_22`


In [27]:
# Computes the projectors onto the non-degenerate eigenvalues
function get_proj_22(op)
    # Get eigenvalues and eigenvectors 
    eig = eigen(op)
    eigenvalues = [round(e, digits=7) for e in eig.values]
    eigenvectors = eig.vectors

    occurred = Set()
    proj_dict = Dict{Float64, Any}()

    # Iterate over eigenvectors to identify the degenerate eigenvalues
    for (idx, projector) in enumerate(eachcol(eigenvectors))
        eigenvalue = eigenvalues[idx]

        if eigenvalue in occurred
            proj_dict[eigenvalue] += projector * projector'
        else
            push!(occurred, eigenvalue)
            proj_dict[eigenvalue] = projector * projector'
        end
    end

    # Sort the non-degeenrate eigenvalues and return the corresponding projectors in order
    sorted_eigenvalues = sort(collect(keys(proj_dict)))
    return [proj_dict[ev] for ev in sorted_eigenvalues]
end;

In [28]:
B22 = [get_proj_22(kron(σ[1], σ[1])), get_proj_22(kron(σ[3], σ[3]))];

A = σt(π/4, -π/4);
C = σt(π/4, -π/4);

### Strong Eavesdropper

#### Guessing A, B, C

In [31]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) # Visibilities of the state
pgs = ones(length(vs))


@threads for (i,v) ∈ collect(enumerate(vs))
    p = p_bilocality(A, B22, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_22, PC, PE3, se_22_strong, level=level, x=1, z=1) # Solution of the SDP problem

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.37523551603555194

#### Guessing A, C

In [32]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) # Visibilities of the state
pgs = ones(length(vs))


@threads for (i,v) ∈ collect(enumerate(vs))
    p = p_bilocality(A, B22, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_22, PC, PE2, se_22_strong, level=level, x=1, z=1) # Solution of the SDP problem

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.3751818925248358

### Double-Eavesdropper

#### Guessing A, B, C

In [34]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) # Visibilities of the state
pgs = ones(length(vs))
@threads for (i,v) ∈ collect(enumerate(vs))
    p = p_bilocality(A, B22, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_22, PC, PE2, PF1, se_22_double_ABC, level=level, x=1, z=1)

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.1874999999999967

#### Guessing A, C

In [35]:
pgs = []
level = 4 # Level of the NPA hierarchy
vs = collect(0.0:0.01:1.0) # Visibilities of the state
pgs = ones(length(vs))
@threads for (i,v) ∈ collect(enumerate(vs))
    p = p_bilocality(A, B22, C, ρ2_bell(v, v)) # Probability distribution p(a,b,c|x,z)
    pg = bilo_pguess(p, PA, PB_22, PC, PE1, PF1, se_22_double_AC, level=level, x=1, z=1)

    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1.0	Pg = 0.375101488099574

# Randomness certification with experimental data

## Experimental data

In this section, we compute the amount of certifiable randomness from the experimental distributions obtained in [this work](https://www.nature.com/articles/ncomms14775), where the (1,4) measurement choices were implemented. Such data have been taken in the form of counts $N_{\mathrm{exp}}(a,b,c|x,z)$ and, due to the experimental imperfections and the finite duration of the measurement, they may present fluctuations that make the experimental probability distribution $p_{\mathrm{exp}}(a,b,c|x,z) = N_{\mathrm{exp}}(a,b,c|x,z) / \sum_{a,b,c} N_{\mathrm{exp}}(a,b,c|x,z)$ to violate the no-signaling constraints, defined as:
\begin{equation}
\sum_a p(a,b,c|x,z) = \sum_a p(a,b,c|x',z)
\end{equation}
\begin{equation}
\sum_c p(a,b,c|x,z) = \sum_c p(a,b,c|x,z')
\end{equation}

For this reason, for each $p_{\mathrm{exp}}$, we find the closest no-signaling distribution by means of a maximumum likelihood problem.

In [25]:


# Maximum likelihood problem to find the no-signaling distribution which is closest to the experimental one
function NS_constraint(NABC)
    model = Model(Ipopt.Optimizer)
    set_attribute(model, "tol", 1e-11)
    set_silent(model)
    
    @variable(model, 0 <= pABC[1:2,1:4,1:2,1:2,1:2])
    
    # AB BC No-Signaling Constraints on observed set variable
    @constraint(model, nsBC, sum(pABC[:,:,:,1,:],dims=1) - sum(pABC[:,:,:,2,:],dims=1) .== 0);
    @constraint(model, nsAB, sum(pABC[:,:,:,:,1],dims=3) - sum(pABC[:,:,:,:,2],dims=3) .== 0);
    
    # Normalization
    @constraint(model, norm, sum(pABC, dims=(1,2,3)) .== 1 )
    
    pAC = sum(pABC, dims=2)
    pA_X = reshape(sum(pABC[:,:,:,:,1], dims=(2,3)), (2,1,1,2,1))
    pC_Z = reshape(sum(pABC[:,:,:,1,:], dims=(1,2)), (1,1,2,1,2))
    
    # Conditional independence between A and C
    @constraint(model, ciAC, pAC - pA_X .* pC_Z .== 0)
    
    @objective(model, Max, sum(NABC .* log.(pABC)))
    
    optimize!(model)

    return value.(pABC)
end;

In [26]:
# Experimental counts N(a,b,c|x,z)
exp_data_cc = npzread("data/exp_data/Punti_exp_counts.npz");
distr_keys = collect(filter(x->startswith(x, "distr"), keys(exp_data_cc)))
distr_keys = sort(distr_keys, by=x->parse(Int, split(x, "_")[end]));

In [ ]:
# Monte Carlo for guessing probabilities

N = length(distr_keys)
n_sample = 50
exp_pgs = ones((N,n_sample))
exp_pg_avg = ones(N)
exp_pg_std = ones(N)
level = 3
@threads for i=1:N
    mc_distr = exp_data_cc[distr_keys[i]]
    for j=1:n_sample
        praw = mc_distr[j,:,:,:,:,1,:]
        p = NS_constraint(praw) # Imposing the no-signaling constraints 

        #pg = bilo_pguess(p, PA, PB_14, PC, PE1, PF1, se_14_double_AC; level=level, x=1, z=1) # Double eavesdropper guessing A and C
        #pg = bilo_pguess(p, PA, PB_14, PC, PE2, PF2, se_14_double_ABC; level=level, x=1, z=1) # Double eavesdropper guessing A, B, and C
        pg = bilo_pguess(p, PA, PB_14, PC, PE4, se_14_strong; level=level, x=1, z=1) # Strong eavesdropper guessing A, B, C
        
        exp_pgs[i,j] = pg
    end
    exp_pg_avg[i] = sum(exp_pgs[i,:])./n_sample # Average guessing probability
    exp_pg_std[i] = sqrt(sum((exp_pgs[i,:] .- exp_pg_avg[i]).^2) / (n_sample - 1)) # Standard deviation
    print("\r $i/$N -> Pguess[$i] = $(exp_pg_avg[i]) +/- $(exp_pg_std[i])")
end

## Theoretical model

The theoretical model we use to simulate the experimental distributions has to take into account two main aspects:
- Imperfect state generation: the quantum state of the photon pairs generated by the SPDC source can be modeled by adding both white and colored noise, obtaining a density matrix of the form
    \begin{equation}
      \rho = v \ket{\Psi^-}\bra{\Psi^-} + (1-v) \left[ \frac{c}{2} (\ket{\Psi^-}\bra{\Psi^-} + \ket{\Psi^+}\bra{\Psi^+}) + (1-c) \frac{\mathbb{1}}{4} \right]
    \end{equation}  
- Imperfect Bell state measurement: due to the partial indistinguishability of the photons incoming at Bob's measurement station, the actual measurement, instead of being a BSM, is described by the following effective POVMs
    \begin{equation}
        \begin{aligned}
           & \ket{\Phi^+}\bra{\Phi^+} \longrightarrow \hat{F}_1 = \frac{1-p}{2} \ket{\Phi^-}\bra{\Phi^-} + \frac{1+p}{2}\ket{\Phi^+}\bra{\Phi^+},\\
           & \ket{\Phi^-}\bra{\Phi^-} \longrightarrow \hat{F}_2 = \frac{1+p}{2} \ket{\Phi^-}\bra{\Phi^-} + \frac{1-p}{2}\ket{\Phi^+}\bra{\Phi^+},\\
           & \ket{\Psi^+}\bra{\Psi^+} \longrightarrow \hat{F}_3 = \frac{1-p}{2} \ket{\Psi^-}\bra{\Psi^-} + \frac{1+p}{2}\ket{\Psi^+}\bra{\Psi^+},\\
           & \ket{\Psi^-}\bra{\Psi^-} \longrightarrow \hat{F}_4 = \frac{1+p}{2} \ket{\Psi^-}\bra{\Psi^-} + \frac{1-p}{2}\ket{\Psi^+}\bra{\Psi^+}.
        \end{aligned}
    \end{equation}


To simulate the [experiment](https://www.nature.com/articles/ncomms14775), we set the parameters $v$ and $p$ to their experimental values $v_{exp}=0.89$ and $c_{exp}=0.33$, while $p$ is changed in the range [0,1].

In [17]:
teo_data = npzread("data/Punti_teo.npz");
viol = teo_data["BRGP_teo"];
distrs = teo_data["distr_teo"];

In [21]:
N = 30
v_exp = 0.89
c_exp = 0.33
p_inds = range(0, 1, 10) # Indistinguishability parameter
level = 3

teo_pgs = ones(N)
teo_BRGP = zeros(N)

@threads for (i, p_ind) ∈ collect(enumerate([1]))
    p = distrs[30,:,:,:,:,1,:]
    #pg = bilo_pguess(p, PA, PB_14, PC, PE1, PF1, se_14_double_AC; level=level, x=1, z=1) # Double eavesdropper guessing A and C
    pg = bilo_pguess(p, PA, PB_14, PC, PE2, PF2, se_14_double_ABC; level=level, x=1, z=1) # Double eavesdropper guessing A, B, and C
    #pg = bilo_pguess(p, PA, PB_14, PC, PE2, se_14_strong; level=level, x=1, z=1) # Strong eavesdropper guessing A, C
        
    teo_pgs[i] = pg
    #teo_BRGP[i] = BRGP(p)
    print("\r $i/$N -> Pguess[$i] = $pg")
end

 1/30 -> Pguess[1] = 0.4580578609703012

# Tilted strategies

These strategies employ different measurements on the external nodes. In particular, we define the following operators, parametrized by $\delta$:
\begin{equation}
    \begin{aligned}
    & A_0 = \sigma_x \qquad C_0 = \sigma_z\\
    & A_1 = \cos{(\delta)} \sigma_x - \sin{(\delta)} \sigma_z \qquad C_1 = \cos{(\delta)} \sigma_z - \sin{(\delta)} \sigma_x
    \end{aligned}
\end{equation}

For what concerns the measurement on the central node we consider two possible strategies:
- (1,4): where Bob performs the standard BSM
- (2,4): where Bob's measurements have 2 settings, consisting in standard BSM and rotated BSM, where the latter is defined as the projections on the basis $\mathcal{B}_{\theta} = \{\cos{\theta} \ket{00} + \sin{\theta} \ket{11}, \cos{\theta} \ket{00} - \sin{\theta} \ket{11}, \cos{\theta} \ket{01} + \sin{\theta} \ket{10}, \cos{\theta} \ket{01} - \sin{\theta} \ket{10} \}$

## (1, 4) measurement strategy

#### Guessing probability for $\delta = \pi/6, v \in [0.7,1]$

In [111]:
M = 20
level = 4

vs = collect(0.7:0.3/M:1.0)
δ = π/6
pgs = ones(length(vs))
@threads for (i,v) ∈ collect(enumerate([1]))
    p = p_bilocality_wbell1(δ, v=v)
    #pg = bilo_pguess(p, PA, PB_14, PC, PE4, se_14_strong; level=level, x=1, z=1) # Strong eavesdropper
    pg = bilo_pguess(p, PA, PB_14, PC, PE2, PF2, se_14_double_ABC, level=level, x=1, z=1) # Double eavesdropper
    print("\r v = $v\tPg = $pg")
    pgs[i] = pg
end

 v = 1	Pg = 0.06286344044997592

#### Guessing probability for $\delta \in [0, \pi/6], v \in [0.7, 1]$

In [ ]:
N, M = 5, 20
level = 4
#PE = PE2
PE = PE4
ebits = Int(log2(size(PE)[1]))

vs = collect(0.7:0.3/M:1.0)
δs = collect(π/10:π/(10*N):π/5)
pgs_wtilted1 = ones((length(vs),length(δs)))
@threads for (i,v) ∈ collect(enumerate(vs))
    for (j,δ) ∈ collect(enumerate(δs))
        p = p_bilocality_wbell1(δ, v=v)
        pg = bilo_pguess(p, PA, PB, PC, PE, se; level=level, x=1, z=1)
        print("\r v = $v\tδ = $δ\tPg = $pg")
        pgs_wtilted1[i,j] = pg
    end
end

## (2,4) measurement strategy

#### Strong-Eavesdropper

In [12]:
# Define projectors corresponding to the three parties measurements
PA = projector('A', 1:2, 1:2, full=true);
PB_24= projector('B', 1:4, 1:2, full=true);
PC = projector('C', 1:2, 1:2, full=true);

In [13]:
# Define projectors corresponding to the eavesdropper measurements
PE2 = projector('D', 1:4, 1, full=true); # E guessing AC
PE4 = projector('D', 1:16, 1, full=true); # E guessing ABC

l = 5 # Order of the product of the scalar variables
con = separability_structure(["A", "B", "C", "D"], ("A","C")); # Separability constraints
vars = generate_projective_scvar(PA, l, swapped=true); # Scalar variables
se_24_strong = ScalarExtension(vars, con); # Scalar Extension

In [ ]:
N, M = 10, 10
δ_interval = [0.1, π/6-.1]
θ_interval = [0, π/2]
level = 4

δs = collect(δ_interval[1]:(δ_interval[2]-δ_interval[1])/N:δ_interval[2])
θs = collect(θ_interval[1]:(θ_interval[2]-θ_interval[1])/M:θ_interval[2])
pgs_tiltedBSM1 = ones((length(δs),length(θs)))

@threads for ((i,δ), (j,θ)) ∈ collect(Iterators.product(enumerate(δs), enumerate(θs)))
    p = p_bilocality_24_wtiltedBSM(δ, θ)
    pg = bilo_pguess(p, PA, PB_24, PC, PE4, se_24_strong; level=level, x=1, y=1, z=1) # Strong eavesdropper
    
    # Note: it is not necessary to compute the guessing probability for the Double eavesdropper scenario with the (2,4) strategy,
    # as the (1,4) strategy already suffices to reach the logical maximum of 4 bits
    
    print("\r δ = $δ\tφ = $φ\tPg = $pg")
    pgs_tiltedBSM1[i,j] = pg
end